### Loading packages

In [1]:
import os
import sys
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling

import sqlite3
from sqlite3 import Error

### Loading [Data from Kaggle competition](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions)

In [2]:
path = './data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

./data/RAW_recipes.csv
./data/interactions_test.csv
./data/interactions_validation.csv
./data/PP_recipes.csv
./data/.~lock.PP_recipes.csv#
./data/PP_users.csv
./data/interactions_train.csv
./data/RAW_interactions.csv


In [3]:
df_recipes_raw = pd.read_csv('./data/RAW_recipes.csv')
df_recipes_cod = pd.read_csv('./data/PP_recipes.csv')

### Examining Data

In [4]:
df_recipes_raw.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
df_recipes_cod.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [6]:
df_recipes = pd.merge(df_recipes_cod, df_recipes_raw, on='id')

In [7]:
df_recipes.iloc[1]["ingredients"]

"['flour', 'oats', 'brown sugar', 'pecans', 'butter', 'eggs', 'solid pack pumpkin', 'sweetened condensed milk', 'cinnamon', 'ginger', 'nutmeg', 'salt']"

In [8]:
df_recipes.iloc[2]["ingredient_ids"]

'[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696, 7946, 5648, 7239, 7705, 7594, 1168, 2683]'

#### Examining the "Tag" field

In [9]:
df_recipes.tags.head()

0    ['weeknight', 'time-to-make', 'course', 'main-...
1    ['60-minutes-or-less', 'time-to-make', 'course...
2    ['30-minutes-or-less', 'time-to-make', 'course...
3    ['15-minutes-or-less', 'time-to-make', 'course...
4    ['30-minutes-or-less', 'time-to-make', 'course...
Name: tags, dtype: object

In [10]:
tags = set([])
for taglist in df_recipes.tags:
    taglist = eval(taglist)
    for tag in taglist:
        tags.add(tag)

In [11]:
len(tags)

530

#### Examining the "Ingredients" field

In [12]:
ingredients = set([])
for ingredient_list in df_recipes.ingredients:
    ingredient_list = eval(ingredient_list)
    for ingredient in ingredient_list:
        ingredients.add(ingredient)

In [13]:
len(ingredients)

13627

### Exporting an abridged dataframe for similarity calculations

In [14]:
useful = ["name", "ingredients", "ingredient_ids"]
df_recipes[useful].head()

,name,ingredients,ingredient_ids
0,aromatic basmati rice rice cooker,"['basmati rice', 'water', 'salt', 'cinnamon st...","[389, 7655, 6270, 1527, 3406]"
1,pumpkin pie a la easy,"['flour', 'oats', 'brown sugar', 'pecans', 'bu...","[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,cheesy tomato soup with potatoes,"['chicken broth', 'water', 'salt', 'black pepp...","[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,mini tacos,"['wonton wrappers', 'hamburger', 'taco seasoni...","[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,rosemary s hanky panky s,"['ground beef', 'ground sausage', 'velveeta ch...","[3484, 6324, 7594, 243]"


In [15]:
df_recipes[useful].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178265 entries, 0 to 178264
Data columns (total 3 columns):
name              178265 non-null object
ingredients       178265 non-null object
ingredient_ids    178265 non-null object
dtypes: object(3)
memory usage: 5.4+ MB


In [17]:
df_recipes.ingredient_ids = df_recipes.ingredient_ids.apply(eval)
df_recipes.ingredients = df_recipes.ingredients.apply(eval)

In [18]:
df_recipes[useful].to_pickle("basic_features.pkl")

### Building a database from the data

In [62]:
db_file = 'recipes.sqlite'

def create_connection(db_file):
    """ 
    create a database connection to the SQLite database
    specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def create_table(conn, create_table_sql):
    """ 
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [71]:
sql_create_ingredients_table = r""" CREATE TABLE IF NOT EXISTS ingredients (
                               id integer PRIMARY KEY,
                               ingredient_name text NOT NULL
                               ); """
 
sql_create_tags_table = r"""CREATE TABLE IF NOT EXISTS tags (
                        id integer PRIMARY KEY,
                        tag_name text NOT NULL
                        );"""
 
sql_create_recipes_table = r"""CREATE TABLE IF NOT EXISTS recipes (
                           id integer PRIMARY KEY,
                           recipe_name text NOT NULL
                           );"""   

sql_create_recipes_tags = r"""CREATE TABLE IF NOT EXISTS recipes_tags (
                           id integer PRIMARY KEY,
                           recipe_id integer NOT NULL,
                           tag_id integer NOT NULL,
                           FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                           FOREIGN KEY (tag_id) REFERENCES tags (id)
                           );""" 

sql_create_recipes_ingredients = r"""CREATE TABLE IF NOT EXISTS recipes_ingredients (
                                 id integer PRIMARY KEY,
                                 recipe_id integer NOT NULL,
                                 ingredient_id integer NOT NULL,
                                 FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                                 FOREIGN KEY (ingredient_id) REFERENCES ingredients (id)
                                 );""" 

# create a database connection
conn = create_connection(db_file)

# create tables
if conn is not None:
    create_table(conn, sql_create_ingredients_table)
    create_table(conn, sql_create_tags_table)
    create_table(conn, sql_create_recipes_table)
    create_table(conn, sql_create_recipes_tags)
    create_table(conn, sql_create_recipes_ingredients)
else:
    print("Error! cannot create the database connection.")

In [72]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, ingredient in enumerate(ingredients):
    cur.execute('INSERT INTO ingredients VALUES(?,?)',(idx, ingredient))

In [73]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, tag in enumerate(tags):
    cur.execute('INSERT INTO tags VALUES(?,?)',(idx, tag))

In [76]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, name in enumerate(df_recipes.name):
    try:
        cur.execute('INSERT INTO recipes VALUES(?,?)',(idx, name))
    except:
        print(name)

nan


In [77]:
cur.execute("select * from tags limit 10;")
for result in cur.fetchall():
    print(result)